In [ ]:
my_fields={'dial': ('Text', data.Field(sequential=True)),
        'emo': ('labels_1', data.Field(sequential=False)),
        'act': ('labels_2', data.Field(sequential=False))}

test = my_TabularDataset.splits(path = working_path, train = 'data_jsonfile/full_data_test.json',
                          fields=my_fields) 
test = sorted(test, key = lambda  x: cal_dialogue(x))
test = sorted(test, key = lambda  x: -len(x.Text)) #reordering training dataset with number of sentences
# low index has much sentence because afterwards we use torch pad_sequence
print(len(test))
dataseq = torch.arange(end = len(train),dtype=torch.int)

In [ ]:
def cal_accuracy(model_predict, real_tag):
    '''
    Args:
        model_predict
            model predicted tags
        real_tag
            real tags
        tag_len
            tag len
        
    Yields:
        accuracy
        
    Example:
        
    real = torch.tensor([  0,   2,   1,   2,   1,   1,   1,   1,   1,   2,   1,   1, 1,   1,  29], device='cuda:0')
    model = [2, 1, 2, 1, 17, 1, 17, 1, 3, 1, 17, 1, 17, 29]
    taglen = len(model)
    
    (npreal[tagseq+1]//4) real emotion
    (model_predict[tagseq]//4) model emotion
    
    (npreal[tagseq+1]%4) real action
    (model_predict[tagseq]%4) model action
    
    emotion err = 0.3076923076923077
    action err = 0.07692307692307693
    accuracy = 0.8076923076923077
    '''
    tag_len = len(model_predict)
    npreal = real_tag.cpu().numpy()
    tagseq = 0
    emotiontag = []
    actiontag = []
    emoerr = 0
    acterr = 0
    while(tagseq < tag_len-1):
      
        emotiontag = np.append(emotiontag, npreal[tagseq+1]//4)
        actiontag = np.append(actiontag, npreal[tagseq+1]%4)
        
        
        if((npreal[tagseq+1]//4) != (model_predict[tagseq]//4)):
            emoerr = emoerr + 1

        
        if((npreal[tagseq+1]%4) != (model_predict[tagseq]%4)):
            acterr = acterr + 1

        tagseq = tagseq + 1

    return(1-(emoerr/tagseq + acterr/tagseq)/2)
    
    

In [ ]:
def test_func(test_data, shared_model , comp_model, dataseq, batch_size):
    """
    Args:
            test_data: 
                test data
                
            shared_model:
                shared model
                
            comp_model:
                comp model
                
            dataseq:
                data sort sequence
                
            batch_size:
                batchsize
            


    Yields:
            accuracy:
                accuracy of all data
    
    """
    

    k = 0
    accuracy = 0
    for batch_data in batchload(test_data, repeat=True, batchsize = batch_size, data_seq = dataseq ):
        #load txt data from jsonfile


        new_dial, new_tag, dial_leng = all_preprocess(shared_model, batch_data)
        #load batch*(dialogue_length*sent_vec(float)) -> new_dial
        #load batch*tag -> new_tag
        #load batch * dial_leng

        
        dummy_input = [make_mask(dial_leng),new_dial]
        unuselist = [new_dial, new_tag, dial_leng]
        del unuselist
        print("tag = ",new_tag[7])
        print("expect = ",comp_model(BATCH_SIZE,dummy_input,seq=7)[1])
        print("accuracy = ", cal_accuracy(comp_model(BATCH_SIZE,dummy_input,seq=7)[1],new_tag[7]))
        
        tag_num = 0
        batch_acc = 0
        while(tag_num < 100):
            batch_acc = batch_acc + cal_accuracy(comp_model(BATCH_SIZE,dummy_input,seq=tag_num)[1],new_tag[tag_num])
            tag_num = tag_num + 1
        batch_acc = batch_acc/100
        accuracy = accuracy + batch_acc
        print(batch_acc)
        if k == int(len(train)/BATCH_SIZE):
            break
            
        k = k + 1
        print(k)

    
    return accuracy/100

In [ ]:

test_func(
    test_data = test, 
    shared_model = sent_to_vextor_bigru_net, 
    comp_model = my_grucrf_model,
    dataseq = dataseq,  
    batch_size = 100)
